# Eigene Fitbit-Daten explorieren

Vorab für diejenigen, die es evtl. noch nicht mitbekommen haben: [Fitbit gehört mittlerweile zu Google](https://www.heise.de/news/Google-schliesst-Fitbit-Uebernahme-ab-5024699.html). D.h., wenn wir Google-Services und FitBit nutzen, können nicht nur wir diese Daten für unsere eigenen Analysen kombinieren, sondern Google kann dies nun auch.

Wie auch für alle anderen Notebooks müssen wir zunächst unsere [eigenen Daten exportieren](https://help.fitbit.com/articles/en_US/Help_article/1133.htm). Hierzu gibt es bei Fitbit grundlegend 2 Optionen:
1. Man kann einen Teil seiner Daten (z.B. zu Aktivitäten und Schlaf) für einen bestimmten Zeitraum (maximal einen Monat) als `CSV`-Dateien exportieren. Wenn wir die Daten für mehrere Monate haben möchten, müssen wir den Export entsprechend mehrfach manuell durchführen.
1. Man kann das komplette Kontoarchiv anfordern. Diese Daten sind wesentlich umfangreicher, decken den gesamten Nutzungszeitraum ab und werden (größtenteils) im `JSON`-Format exportiert.

Wir werden in diesem Notebook hauptsächlich mit Daten aus dem vollständigen Kontoarchiv arbeiten, aber auch kurz auf das Einlesen der einzeln exportierbaren `CSV`-Dateien eingehen.

Wenn Sie die Datei mit dem Kontoarchiv heruntergeladen haben, müssen die diese entpacken. Die Daten sollten dann im Ordner `MyFitbitData/Ihr_Name` liegen. Wir werden uns in diesem Notebook auf Daten zu Schritten und Schlaf fokussieren. Diese sind in den Unterordnern `Physical Activity` und `Sleep` zu finden. 

Auch die FitBit-Daten müssen wir hier hochladen, damit wir im Notebook mit ihnen arbeiten können. Da diese mehrere Dateien sind und um zumindest ein bisschen Ordnung zu halten, was v.a. sinnvoll ist, wenn wir (in einer Sitzung) mit mehreren Notebooks und Datentypen, erstellen wir einen neuen Unterordner für die Fitbit-Daten im `data`-Ordner.

In [ ]:
dir.create("./data/fitbit")

Nun können wir die Dateien in diesen neu erstellten Ordner laden. Wir nutzen in diesem Notebook die Daten zu den gezählten Schritten (Steps) sowie zum Schlag (Sleep). Die benötigten `JSON`-Files sind in den o.g. Ordnern im exportierten Archiv zu finden. Die Namen der Dateien zu den Schritten beginnen mit "steps-" und bei den Schlaf-Daten beginnen die Dateinamen mit "sleep-", jeweils gefolgt von einem Datum im Format JJJJ-MM-TT.

Wenn Sie auch (oder nur) die `CSV`-Dateien mit einer Auswahl der Daten für (jeweils) maximal einen Monat exportiert haben, können Sie diese ebenso hier in den Ordner `data/fitbit` hochladen. Die Namen dieser Dateien sollten dem Muster "fitbit_export_JJJJMMTT.csv" entsprechen.

**Hinweis**: Wenn Sie Fitbit schon lange nutzen, kann die Anzahl der Dateien recht groß sein. In diesem Fall können der Upload sowie das Einlesen (im Code weiter unten) etwas länger dauern. Eine Möglichkeit, dem entgegenzuwirken, ist, nur einen Teil der jeweiligen Daten/Dateien zu nutzen (und somit den Zeitraum einzugrenzen).

**Zur Erinnerung**: Ihre persönliche Kopie des Notebooks sowie alle Dateien, die Sie hochladen, werden am Ende der Nutzungssitzung gelöscht (es sei denn, Sie haben und nutzen einen Account für GESIS Notebooks). Wenn Sie aber ganz "auf Nummer sicher gehen" wollen, können Sie die Dateien mit Ihren Fitbit-Daten über den File Explorer auf der linken Seite nach dem Durcharbeiten dieses Notebooks auch manuell löschen: Rechtsklick auf den Dateinamen und dann *Delete* auswählen.

# Pakete laden

Bevor wir mit den Daten arbeiten können laden wir die `R`-Pakete, die wir dafür brauchen.

In [ ]:
library(readr)
library(jsonlite)
library(dplyr)
library(lubridate)
library(ggplot2)
library(scales)
library(stringr)
library(purrr)

## Daten einlesen & aufbereiten

### `CSV`-Dateien mit einer Datenauswahl pro Monat

**Hinweis**: Wenn man beim direkten `CSV`-Export im eigenen FitBit-Account mehrere Datentypen auf einmal auswählt, werden diese mit verschiedenen Spalten-/Variablennamen untereinander in eine `CSV`-Datei geschrieben. Dies ist ungünstig, um die Daten z.B. mit `R` oder `Python` zu öffnen und zu bearbeiten. Die Datentypen sollten daher als einzelne `CSV`-Dateien exportiert werden (z.B. eine für Sleep und eine für Activities). 

**NB**: Ich habe die beiden `CSV`-Dateien umbenannt, damit der Dateiname anzeigt, welche Art von Daten diese enthalten. Für Ihre Daten müssen Sie die Dateinamen in der nachfolgenden Code-Zelle entsprechend anpassen.

Wie zuvor gesagt, beschränken wir uns in diesem Notebook auf die Daten zu Schlaf und Aktivitäten (Schritte) und werden die eigentliche Exploration im weiteren Verlauf mit den Daten aus dem kompletten Kontoarchiv durchführen.

In [ ]:
activities_122020 <- read_csv("./data/fitbit/fitbit_export_20210112_activities.csv",
                              locale = locale(grouping_mark = "."),
                              skip = 1)

sleep_122020 <- read_csv("./data/fitbit/fitbit_export_20210112_sleep.csv",
                              skip = 1)

sleep_122020 <- sleep_122020 %>% 
  mutate(fell_asleep = parse_date_time(Startzeit, 
                                       orders = "dmYIMp",
                                       tz = "Europe/Berlin"),
         woke_up = parse_date_time(Endzeit, 
                                       orders = "dmYIMp",
                                       tz = "Europe/Berlin"))

Da wir im weiteren Verlauf des Notebooks mit den Daten aus den `JSON`-Files aus dem vollständigen Kontoarchiv arbeiten, schauen wir uns für die `CSV`-Exporte nur die Variablennamen sowie die jeweils ersten zehn Fälle an.

In [ ]:
names(activities_122020)
head(activities_122020, 10)

names(sleep_122020)
head(sleep_122020, 10)

### Daten aus dem Kontoarchiv

Nun lesen wir die Daten zu Schlaf und Schritten aus dem kompletten Kontoarchiv ein. Da diese auf mehrere `JSON`-Dateien verteilt sind, müssen wir diese aus dem entsprechenden Verzeichnis einlesen und kombinieren.

Sidenote: Anstatt die einzelnen `JSON`-Files mit `R` einzulesen und zu verbinden, ist es auch möglich, das Tool [Fitbit Data JSON to .csv File Converter](https://iccir919.github.io/fitbit-json-to-csv/index.html) zu nutzen. Diese läuft lokal im Browser und verbindet die `JSON`-Dateien zu (großen) `CSV`-Dateien (**NB**: Bei mir hat das nicht 100%ig funktioniert: Von den 18 Dateien, die ich in meinem Test mit diesem Tool zusammenfügen wollte, wurden nur 16 verbunden, weshalb ein Teil der Daten im entstandenen `CSV`- File fehlte). Das Tool übernimmt neben der Verknüpfung der Dateien und der Konvertierung des Formats auch die Erstellung korrekter Zeitstempel. Die resultierenden `CSV`-Dateien kann man dann bspw. in `R` wie in der Code-Zelle oben mit dem Befehl `read_csv` aus dem `dplyr`-Paket einlesen.

In [ ]:
path <- "./data/fitbit"
files <- dir(path, pattern = "*.json")
files_steps <- str_subset(files, "steps")

steps <- files_steps %>%
  map_df(~fromJSON(file.path(path, .), flatten = TRUE))

steps <- steps %>% 
  mutate(datetime = mdy_hms(dateTime, tz = "GMT"),
         datetime = with_tz(datetime, tzone = "Europe/Berlin"),
         date = as_date(datetime),
         wday = wday(date, week_start = 1, label = T),
         value = as.numeric(value))

files_sleep <- str_subset(files, "sleep")

sleep <- files_sleep %>%
  map_df(~fromJSON(file.path(path, .), flatten = TRUE))

sleep <- sleep %>%
  distinct(startTime, .keep_all = T) %>% 
  select(dateOfSleep, minutesAsleep) %>% 
  mutate(date = ymd(dateOfSleep),
         wday = wday(date, week_start = 1, label = T))

Wie sehen die Daten aus?

In [ ]:
names(steps)
head(steps, 10)

names(sleep)
head(sleep, 10)

## Schritte

Welchen Zeitraum decken die Daten ab?

In [ ]:
min(steps$date)
max(steps$date)

### Durchschnittliche Anzahl von Schritten pro Wochentag

In [ ]:
options(scipen = 10)

steps %>% 
  group_by(date) %>% 
  summarize(steps = sum(value)) %>% 
  ungroup() %>% 
  mutate(wday = wday(date, week_start = 1, label = T)) %>% 
  group_by(wday) %>% 
  summarize(avg_steps = mean(steps, na.rm = T)) %>% 
  ungroup() %>% 
  ggplot(aes(x = wday, y = avg_steps)) +
  geom_bar(stat = "identity") +
  scale_y_continuous(expand = expansion(mult=c(0,0.1))) +
  labs(title = "Average number of steps per weekday as tracked by Fitbit",
       x = "Weekday",
       y = "Steps")

### Schritte pro Tag

In [ ]:
steps_per_day <- steps %>% 
  group_by(date) %>% 
  summarize(steps = sum(value)) %>% 
  ungroup()

summary(steps_per_day$steps) 

Welches waren die aktivsten Tage?

In [ ]:
steps_per_day %>% 
  arrange(-steps) %>% 
  head(10)

### Schritte pro Monat

In [ ]:
steps_per_day %>%
  mutate(time_floor = floor_date(date, unit = "1 month")) %>%
  group_by(time_floor) %>%
  summarize(steps = sum(steps)) %>% 
  ggplot(aes(x = as.factor(str_sub(as.character(time_floor), end = -4)), y = steps)) +
  geom_bar(stat = "identity") +
  scale_y_continuous(expand = expansion(mult=c(0,0.1))) +
  labs(title = "Number of steps taken per month",
       x = "Month",
       y = "Number of steps") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

## Schlaf

Welchen Zeitraum decken die Daten ab?

In [ ]:
min(sleep$date)
max(sleep$date)

### Durchschnittliche Schlafdauer pro Wochentag

In [ ]:
sleep %>% 
  group_by(date) %>% 
  mutate(hrs_of_sleep = sum(minutesAsleep)/60) %>% 
  ungroup() %>% 
  group_by(wday) %>% 
  summarize(avg_hrs_sleep = mean(hrs_of_sleep, na.rm = T)) %>% 
  ungroup() %>% 
  ggplot(aes(x = wday, y = avg_hrs_sleep)) +
  geom_bar(stat = "identity") +
  scale_y_continuous(expand = expansion(mult=c(0,0.1))) +
  labs(title = "Average hours of sleep per weekday as tracked by Fitbit",
       x = "Weeday",
       y = "Average hours of sleep")

### Schlafstunden pro Tag

Mittlere, minimale und maximale aufgezeichnete Anzahl an Schlafstunden.

In [ ]:
sleep_day <- sleep %>%
  group_by(date) %>%
  summarize(sleep_tot=sum(minutesAsleep)/60)

mean(sleep_day$sleep_tot)

min(sleep_day$sleep_tot)
max(sleep_day$sleep_tot)

Für wie viele Tage liegen Daten vor?

In [ ]:
n_distinct(sleep_day$date)

### Verteilung der Schlafstunden

**NB**: Die Anzahl der Tage im Subtitel des Plots können bzw. sollten Sie entsprechend des Outputs der vorherigen Code-Zelle anpassen.

In [ ]:
sleep_day %>% 
ggplot(aes(x = sleep_tot))+
  geom_histogram(binwidth = 0.5) +
  scale_x_continuous(breaks = seq(0, 12 , 1),
                     expand = expansion(mult=c(0,0))) +
  scale_y_continuous(expand = expansion(mult=c(0,0.1))) +
  coord_cartesian(xlim=c(0, 12)) +
  labs(title = "How many hours do I sleep per day?", 
       subtitle = "Data from 372 days of wearing a Fitbit",
       x = "Hours of sleep",
       y = "Count") +
  theme_minimal()

## Ausblick

In den Fitbit-Daten stecken natürlich noch viel mehr Informationen drin. Wir haben uns davon hier nur einen sehr kleinen Ausschnitt angeschaut und auch mit den Daten, die wir in diesem Notebook explorieren, sind noch viele andere Analysen möglich. Ähnlich wie auch für die anderen Daten(typen) gibt es im Netz einige Tutotials zur Exploration von Fitbit-Daten (ein sehr umfangreiches zu Schlafdaten gibt es z.B. [hier](https://thelittledataset.com/2020/02/04/fitbit-sleep/)).

Wie bie Twitter ist es auch möglich, die eigenen Daten über die API von Fitbit zu sammeln. Hierzu eignet sich das `R`-Paket [`fitbitr`](https://github.com/teramonagi/fitbitr). Ebenso wie für Twitter-Daten sind für die Nutzung der API ein Developer Account sowie die Erstellung einer "App" nötig. Wie dies geht, wird in der Dokumentation des `fitbitr`-Pakets (auf GitHub) beschrieben.